In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 필요한 라이브러리 호출, 데이터 읽기

In [ ]:
import numpy as np
import pandas as pd
import gc
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
# 화면 크기 설정
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
# listdir를 사용하여 디렉터리 리스트 확인
print(os.listdir("../input/home-credit-default-risk"))

Training data와 test data 읽기

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

In [ ]:
# training 데이터의 shape 확인
app_train.shape

In [ ]:
# testing 데이터의 shape 확인
app_test.shape

### app_train에 대한 EDA

In [ ]:
# app_train의 전체 컬럼 개수는 122개 이며, 이 중에서 결측치가 있는 컬럼 개수는 67개 
app_train.isnull().sum()

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
# TARGET은 불균형한 데이터
app_train['TARGET'].astype(int).plot.hist();

각 feature들에 대해 TARGET값 분포도 비교 시각화

In [ ]:
def show_hist_by_target(df, columns):
    cond_1 = (df['TARGET'] == 1)
    cond_0 = (df['TARGET'] == 0)
    
    for column in columns:
        fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=ax[0][0])
        sns.distplot(df[cond_1][column], label='1', color='red', ax=ax[0][1])
        sns.distplot(df[cond_0][column], label='0', color='blue', ax=ax[0][1])

In [ ]:
# Feature Importances 에서 중요하게 나왔던 칼럼만 선택
columns = ['AMT_INCOME_TOTAL','AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
           'DAYS_REGISTRATION', 'DAYS_LAST_PHONE_CHANGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'EXT_SOURCE_1', 
           'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 
           'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

show_hist_by_target(app_train, columns)

In [ ]:
# DAYS_BIRTH의 절대값과 TARGET변수와의 상관계수
app_train['DAYS_BIRTH']=abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

In [ ]:
plt.style.use('fivethirtyeight')

# 고객 나이에 대한 히스토그램 분포 확인
plt.hist(app_train['DAYS_BIRTH']/365, edgecolor='k',bins=25)
plt.title('Age of Client');
plt.xlabel('Age (years)');
plt.ylabel('Count');

In [ ]:
plt.figure(figsize=(10,8))

# 제때 대출을 상환하는 고객의 나이 plot (TARGET=0)
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target==0')

# 제때 대출을 상환하지못하는 고객의 나이 plot (TARGET=1)
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target==1')

plt.xlabel('Age(years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

# --> 30대 고객이 제때 대출을 상환하는 경우가 가장 많

범주형 feature를 TARGET 유형에 따라 비교

In [ ]:
object_columns = app_train.dtypes[app_train.dtypes == 'object'].index.tolist()
object_columns

In [ ]:
sns.catplot(x='CODE_GENDER', col='TARGET', data=app_train, kind='count')

In [ ]:
def show_category_by_target(df, columns):
    for column in columns:
        chart = sns.catplot(x=column, col='TARGET', data=df, kind='count')
        chart.set_xticklabels(rotation=65)

show_category_by_target(app_train, object_columns)

In [ ]:
# 대출 횟수 대비 연체 비율이 남성이 여성보다 높음 --> 이를 value_counts()로 확인

cond_1 = (app_train['TARGET'] == 1)
cond_0 = (app_train['TARGET'] == 0)
print(app_train['CODE_GENDER'].value_counts()/app_train.shape[0])
print('\n연체인 경우\n',app_train[cond_1]['CODE_GENDER'].value_counts()/app_train[cond_1].shape[0])
print('\n연체가 아닌 경우\n',app_train[cond_0]['CODE_GENDER'].value_counts()/app_train[cond_0].shape[0])

training 데이터와 testing 데이터를 합쳐서 preprocessing 수행

In [ ]:
apps = pd.concat([app_train, app_test])
apps.shape

In [ ]:
apps['TARGET'].value_counts(dropna=False)

Object feature들을 Label Encoding

In [ ]:
apps.info()

In [ ]:
apps.dtypes.index

In [ ]:
object_columns = apps.dtypes[apps.dtypes == 'object'].index.tolist()

In [ ]:
apps['CODE_GENDER'] = pd.factorize(apps['CODE_GENDER'])[0]

In [ ]:
# Label Encoding으로 인해 object 개수가 16개 -> 15개로 줄었음
apps.info()

In [ ]:
object_columns = apps.dtypes[apps.dtypes == 'object'].index.tolist()

for column in object_columns:
    apps[column] = pd.factorize(apps[column])[0]

Null값 일괄 변환

In [ ]:
apps.isnull().sum().head(100)

In [ ]:
# 모든 컬럼들의 Null값을 -999로 변환
apps = apps.fillna(-999)

In [ ]:
apps.isnull().sum().head(100)

training 데이터와 test 데이터 다시 분리

In [ ]:
apps_train = apps[apps['TARGET'] != -999]
apps_test = apps[apps['TARGET'] == -999]
apps_train.shape, app_test.shape

In [ ]:
apps_test = apps_test.drop('TARGET', axis=1, inplace=False)
apps_test.shape

학습 데이터를 검증 데이터로 분리하고 LGBM Classifier로 학습 수행

In [ ]:
from sklearn.model_selection import train_test_split

ftr_app = apps_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = app_train['TARGET']

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

학습된 Classifier를 이용하여 테스트 데이터 예측

In [ ]:
pred = clf.predict_proba(apps_test.drop(['SK_ID_CURR'], axis=1))[:, 1]
app_test['TARGET'] = pred
app_test['TARGET'].head()

In [ ]:
app_test[['SK_ID_CURR', 'TARGET']].to_csv('app_baseline_01.csv', index=False)